In [19]:
%load_ext autoreload
import os
import sys
import rootutils
import pandas as pd
sys.path.append('/home/filip-marcus/ESGPT_new/EventStreamGPT/')

from pathlib import Path

import polars as pl
from EventStream.data.dataset_polars import Dataset

# root = rootutils.setup_root(os.path.abspath(""), dotenv=True, pythonpath=True, cwd=False)
 # CHECK IF YOU WANT NEW OR OLD DIR

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def has_event_type(type_str: str) -> pl.Expr:
    event_types = pl.col("event_type").cast(pl.Utf8).str.split("&")
    return event_types.list.contains(type_str)


### Test ECOM

In [1]:
COHORT_NAME = "test_ecom"
PROJECT_DIR = Path('/home/filip-marcus/models/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

NameError: name 'Path' is not defined

In [4]:
events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/test_ecom/events_df.parquet...
shape: (6_024, 4)
┌──────────┬────────────┬─────────────────────┬────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type │
│ ---      ┆ ---        ┆ ---                 ┆ ---        │
│ u16      ┆ u16        ┆ datetime[μs]        ┆ cat        │
╞══════════╪════════════╪═════════════════════╪════════════╡
│ 3        ┆ 1          ┆ 2020-09-24 12:19:57 ┆ view       │
│ 25       ┆ 16         ┆ 2020-09-24 15:06:38 ┆ view       │
│ 31       ┆ 16         ┆ 2020-09-24 15:41:46 ┆ view       │
│ 38       ┆ 16         ┆ 2020-09-25 09:08:01 ┆ view       │
│ …        ┆ …          ┆ …                   ┆ …          │
│ 9869     ┆ 5601       ┆ 2020-09-26 22:23:35 ┆ view       │
│ 9892     ┆ 5619       ┆ 2020-09-26 22:54:36 ┆ view       │
│ 9916     ┆ 5637       ┆ 2020-09-26 23:30:51 ┆ view       │
│ 9918     ┆ 5637       ┆ 2020-09-26 23:40:27 ┆ view       │
└──────────

In [ ]:
%%time

event_types = ['view', 'cart','purchase']

# Create boolean flags for each event type in the event sequence
event_flags = [has_event_type(event).alias(f"is_{event.lower()}") for event in event_types]


[<polars.expr.expr.Expr object at 0x71b3584ff820>, <polars.expr.expr.Expr object at 0x71b3584fee60>, <polars.expr.expr.Expr object at 0x71b3584fdb40>]
CPU times: user 379 μs, sys: 90 μs, total: 469 μs
Wall time: 470 μs


#### Dummy task

In [6]:
# Create event data and calculate class distribution for each sequence
class_distribution_df = (
    events_df.with_columns(*event_flags)  # Add all event flags
    .filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        [
            pl.when(pl.col(f"is_{event.lower()}")).then(pl.lit(1)).otherwise(pl.lit(0)).alias(f"{event.lower()}_class")
            for event in event_types
        ]
    )
    .groupby("subject_id")
    .agg(
        # Calculate the class distribution per subject_id
        *[
            (pl.col(f"{event.lower()}_class").sum() / pl.col(f"{event.lower()}_class").count()).alias(f"{event.lower()}_dist")
            for event in event_types
        ],
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    )
)

# Task dataframe: Select relevant columns for class distribution
class_distribution_df_all = class_distribution_df.select(["subject_id", "start_time", "end_time"] + [f"{event.lower()}_dist" for event in event_types])


# DUMMY SECTION
class_distribution_df_all = class_distribution_df_all.select(class_distribution_df_all.columns[:-2])

last_col = class_distribution_df_all.columns[-1]

# Modify the last column based on the condition
class_distribution_df_all = class_distribution_df_all.with_columns(
    pl.when(pl.col(last_col) > 0.0)
    .then(1)
    .otherwise(0)
    .alias("label")
)
class_distribution_df_all = class_distribution_df_all.select(["subject_id", "start_time","end_time", "label"])
# END DUMMY SECTION

print(class_distribution_df_all.collect())

# Save the class distribution DataFrame
# class_distribution_df_all.collect().write_parquet(TASK_DF_DIR / "task_df_ecom_cls_test.parquet")
# print(f"saved task_df to {TASK_DF_DIR}")

shape: (1_727, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────┐
│ subject_id ┆ start_time          ┆ end_time            ┆ label │
│ ---        ┆ ---                 ┆ ---                 ┆ ---   │
│ u16        ┆ datetime[μs]        ┆ datetime[μs]        ┆ i32   │
╞════════════╪═════════════════════╪═════════════════════╪═══════╡
│ 1          ┆ 2020-09-24 11:57:26 ┆ 2020-09-24 12:19:57 ┆ 1     │
│ 8          ┆ 2020-09-24 11:58:25 ┆ 2020-09-24 12:07:26 ┆ 1     │
│ 10         ┆ 2020-09-24 11:58:34 ┆ 2020-09-24 12:00:59 ┆ 1     │
│ 11         ┆ 2020-09-24 11:59:26 ┆ 2020-09-24 12:02:12 ┆ 1     │
│ …          ┆ …                   ┆ …                   ┆ …     │
│ 5658       ┆ 2020-09-27 00:38:10 ┆ 2020-09-27 00:39:00 ┆ 1     │
│ 5659       ┆ 2020-09-27 00:49:03 ┆ 2020-09-27 00:57:44 ┆ 1     │
│ 5661       ┆ 2020-09-27 00:54:25 ┆ 2020-09-27 00:57:28 ┆ 1     │
│ 5662       ┆ 2020-09-27 00:56:08 ┆ 2020-09-27 01:00:16 ┆ 1     │
└────────────┴─────────────────────┴────────

#### Class dist

In [7]:

# Create event data and calculate class distribution for each sequence
class_distribution_df = (
    events_df.with_columns(*event_flags)  # Add all event flags
    .filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        [
            pl.when(pl.col(f"is_{event.lower()}")).then(pl.lit(1)).otherwise(pl.lit(0)).alias(f"{event.lower()}_class")
            for event in event_types
        ]
    )
    .groupby("subject_id")
    .agg(
        # Calculate the class distribution per subject_id
        *[
            (pl.col(f"{event.lower()}_class").sum() / pl.col(f"{event.lower()}_class").count()).alias(f"{event.lower()}_dist")
            for event in event_types
        ],
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    )
)

# Task dataframe: Select relevant columns for class distribution
class_distribution_df_all = class_distribution_df.select(["subject_id", "start_time", "end_time"] + [f"{event.lower()}_dist" for event in event_types])


print(class_distribution_df_all.collect())

# Save the class distribution DataFrame
class_distribution_df_all.collect().write_parquet(TASK_DF_DIR / "task_df_ecom_cls_test.parquet")
print(f"saved task_df to {TASK_DF_DIR}")


shape: (1_727, 6)
┌────────────┬─────────────────────┬─────────────────────┬───────────┬───────────┬───────────────┐
│ subject_id ┆ start_time          ┆ end_time            ┆ view_dist ┆ cart_dist ┆ purchase_dist │
│ ---        ┆ ---                 ┆ ---                 ┆ ---       ┆ ---       ┆ ---           │
│ u16        ┆ datetime[μs]        ┆ datetime[μs]        ┆ f64       ┆ f64       ┆ f64           │
╞════════════╪═════════════════════╪═════════════════════╪═══════════╪═══════════╪═══════════════╡
│ 1          ┆ 2020-09-24 11:57:26 ┆ 2020-09-24 12:19:57 ┆ 1.0       ┆ 0.0       ┆ 0.0           │
│ 8          ┆ 2020-09-24 11:58:25 ┆ 2020-09-24 12:07:26 ┆ 1.0       ┆ 0.0       ┆ 0.0           │
│ 10         ┆ 2020-09-24 11:58:34 ┆ 2020-09-24 12:00:59 ┆ 1.0       ┆ 0.0       ┆ 0.0           │
│ 11         ┆ 2020-09-24 11:59:26 ┆ 2020-09-24 12:02:12 ┆ 1.0       ┆ 0.0       ┆ 0.0           │
│ …          ┆ …                   ┆ …                   ┆ …         ┆ …         ┆ …       

### Classification if event sequence contains 0: only view, 1: view and cart or 2: view,cart and purchase

In [17]:
# Create event data and calculate class distribution for each sequence
class_distribution_df = (
    events_df.with_columns(*event_flags)  # Add all event flags
    .filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        [
            pl.when(pl.col(f"is_{event.lower()}")).then(pl.lit(1)).otherwise(pl.lit(0)).alias(f"{event.lower()}_class")
            for event in event_types
        ]
    )
    .groupby("subject_id")
    .agg(
        # Calculate the class distribution per subject_id
        *[
            (pl.col(f"{event.lower()}_class").sum() / pl.col(f"{event.lower()}_class").count()).alias(f"{event.lower()}_dist")
            for event in event_types
        ],
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    )
)

# Task dataframe: Select relevant columns for class distribution
class_distribution_df_all = class_distribution_df.select(["subject_id", "start_time", "end_time"] + [f"{event.lower()}_dist" for event in event_types])

# create col with label: 1 if purchase dist > 0, 0 otherwise
class_distribution_df_all = class_distribution_df_all.with_columns(
    pl.when(pl.col("view_dist") == 1.0)
    .then(pl.lit(0))
    .when(pl.col("purchase_dist") > 0.0)
    .then(pl.lit(2))
    .otherwise(pl.lit(1))
    .alias("label")
)

class_distribution_df_all = class_distribution_df_all.drop(["view_dist", "cart_dist", "purchase_dist"])
print(class_distribution_df_all.collect())

label_counts = class_distribution_df_all.select(pl.col("label").value_counts())

print(label_counts.collect())  # Print the count of each label

# Save the class distribution DataFrame
class_distribution_df_all.collect().write_parquet(TASK_DF_DIR / "task_df_ecom_cls_purchase.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

shape: (1_727, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────┐
│ subject_id ┆ start_time          ┆ end_time            ┆ label │
│ ---        ┆ ---                 ┆ ---                 ┆ ---   │
│ u16        ┆ datetime[μs]        ┆ datetime[μs]        ┆ i32   │
╞════════════╪═════════════════════╪═════════════════════╪═══════╡
│ 1          ┆ 2020-09-24 11:57:26 ┆ 2020-09-24 12:19:57 ┆ 0     │
│ 8          ┆ 2020-09-24 11:58:25 ┆ 2020-09-24 12:07:26 ┆ 0     │
│ 10         ┆ 2020-09-24 11:58:34 ┆ 2020-09-24 12:00:59 ┆ 0     │
│ 11         ┆ 2020-09-24 11:59:26 ┆ 2020-09-24 12:02:12 ┆ 0     │
│ …          ┆ …                   ┆ …                   ┆ …     │
│ 5658       ┆ 2020-09-27 00:38:10 ┆ 2020-09-27 00:39:00 ┆ 0     │
│ 5659       ┆ 2020-09-27 00:49:03 ┆ 2020-09-27 00:57:44 ┆ 0     │
│ 5661       ┆ 2020-09-27 00:54:25 ┆ 2020-09-27 00:57:28 ┆ 1     │
│ 5662       ┆ 2020-09-27 00:56:08 ┆ 2020-09-27 01:00:16 ┆ 0     │
└────────────┴─────────────────────┴────────

#### Masked random event impute 'event_type'

In [36]:

print(pl.col("event_type").sample(1))
# Create event data and calculate class distribution for each sequence
masked_event_type_df = (
    events_df.with_columns(*event_flags)  # Add all event flags
    .filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        [
            pl.when(pl.col(f"is_{event.lower()}")).then(pl.lit(1)).otherwise(pl.lit(0)).alias(f"{event.lower()}_class")
            for event in event_types
        ]
    )
    .groupby("subject_id")
    .agg(
        # Calculate the class distribution per subject_id
        *[
            # pl.col("event_id").sample(1),
            pl.col("event_type").sample(1).first().alias("masked_event_type"),
        ],
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    )
)

# Task dataframe: Select relevant columns for class distribution
masked_event_type_df_all = masked_event_type_df.select(["subject_id", "start_time", "end_time", "masked_event_type"] )


print(masked_event_type_df_all.collect())

# Save the class distribution DataFrame
# masked_event_type_df_all.collect().write_parquet(TASK_DF_DIR / "task_df_ecom_masked_event_type.parquet")
# print(f"saved task_df to {TASK_DF_DIR}")


col("event_type").sample_n()
shape: (1_727, 4)
┌────────────┬─────────────────────┬─────────────────────┬───────────────────┐
│ subject_id ┆ start_time          ┆ end_time            ┆ masked_event_type │
│ ---        ┆ ---                 ┆ ---                 ┆ ---               │
│ u16        ┆ datetime[μs]        ┆ datetime[μs]        ┆ cat               │
╞════════════╪═════════════════════╪═════════════════════╪═══════════════════╡
│ 1          ┆ 2020-09-24 11:57:26 ┆ 2020-09-24 12:19:57 ┆ view              │
│ 8          ┆ 2020-09-24 11:58:25 ┆ 2020-09-24 12:07:26 ┆ view              │
│ 10         ┆ 2020-09-24 11:58:34 ┆ 2020-09-24 12:00:59 ┆ view              │
│ 11         ┆ 2020-09-24 11:59:26 ┆ 2020-09-24 12:02:12 ┆ view              │
│ …          ┆ …                   ┆ …                   ┆ …                 │
│ 5658       ┆ 2020-09-27 00:38:10 ┆ 2020-09-27 00:39:00 ┆ view              │
│ 5659       ┆ 2020-09-27 00:49:03 ┆ 2020-09-27 00:57:44 ┆ view              │
│ 566

### Eneryield TTE

In [3]:
COHORT_NAME = "eneryield_combined"
PROJECT_DIR = Path('/home/filip-marcus/models/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())


Loading events from /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_combined/events_df.parquet...
shape: (332_042, 4)
┌──────────┬────────────┬─────────────────────┬─────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type  │
│ ---      ┆ ---        ┆ ---                 ┆ ---         │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat         │
╞══════════╪════════════╪═════════════════════╪═════════════╡
│ 5        ┆ 2          ┆ 2025-01-24 07:07:57 ┆ disturbance │
│ 6        ┆ 2          ┆ 2025-01-24 12:54:08 ┆ disturbance │
│ 22       ┆ 5          ┆ 2025-01-21 22:24:30 ┆ disturbance │
│ 31       ┆ 5          ┆ 2025-01-24 12:54:08 ┆ disturbance │
│ …        ┆ …          ┆ …                   ┆ …           │
│ 332039   ┆ 1406       ┆ 2020-10-06 12:11:11 ┆ disturbance │
│ 332046   ┆ 1406       ┆ 2020-10-08 07:51:45 ┆ disturbance │
│ 332049   ┆ 1407       ┆ 2020-10-06 12:11:02 ┆ disturbance │
│ 332054   ┆ 1407       ┆ 2020-10-07 00:52:12 ┆ 

In [4]:
tte_df = (
    events_df.filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        pl.col("timestamp").shift(-1).over("subject_id").alias("next_event_time")
    )
    .with_columns(
        # Calculate the time difference and convert it to seconds using .seconds()
        (pl.col("next_event_time") - pl.col("timestamp")).dt.seconds().alias("label")  # Duration in seconds
    )
    .groupby("subject_id")
    .agg(
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time"),
        # Apply the function to get the second-to-last label value in seconds
        pl.col("label").apply(lambda x: x[-2] if len(x) >= 2 else None)
    )
)


# tte_df = tte_df.with_columns(pl.lit(1).alias("label"))
tte_df = tte_df.with_columns(pl.col("label").cast(pl.Float64))

tte_df.collect()
# print(tte_df.dtypes)


subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],f64
0,2025-02-01 13:51:48,2025-02-01 13:53:09,81.0
1,2025-02-01 13:51:48,2025-02-01 13:53:09,81.0
2,2025-01-24 07:02:57,2025-01-24 12:54:08,20771.0
3,2025-01-23 23:10:07,2025-01-24 12:54:08,20771.0
4,2025-01-22 06:21:20,2025-01-24 12:54:08,20771.0
5,2025-01-21 09:58:36,2025-01-24 12:54:08,20771.0
6,2025-01-20 04:26:17,2025-01-24 12:54:08,20771.0
7,2025-01-20 04:26:17,2025-01-24 12:54:08,20771.0
8,2025-01-20 04:26:17,2025-01-24 12:54:08,20771.0


In [12]:
# Save the task_df
  
tte_df.collect().write_parquet(TASK_DF_DIR / "task_df_eneryield_tte.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_combined/task_dfs


### Eneryield classification if interruption in this event sequence (subject)

In [ ]:
COHORT_NAME = "eneryield_event_type"
PROJECT_DIR = Path('/home/filip-marcus/models/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/events_df.parquet...
shape: (330_052, 4)
┌──────────┬────────────┬─────────────────────┬─────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type  │
│ ---      ┆ ---        ┆ ---                 ┆ ---         │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat         │
╞══════════╪════════════╪═════════════════════╪═════════════╡
│ 5        ┆ 3          ┆ 2025-01-24 07:07:57 ┆ unbalance_i │
│ 14       ┆ 5          ┆ 2025-01-23 23:10:07 ┆ unbalance_i │
│ 19       ┆ 6          ┆ 2025-01-21 09:58:36 ┆ unbalance_i │
│ 23       ┆ 6          ┆ 2025-01-22 06:22:03 ┆ transient   │
│ …        ┆ …          ┆ …                   ┆ …           │
│ 330014   ┆ 1408       ┆ 2020-10-07 00:52:12 ┆ unbalance_u │
│ 330022   ┆ 1408       ┆ 2020-10-09 17:50:47 ┆ VD          │
│ 330029   ┆ 1408       ┆ 2020-10-11 11:32:57 ┆ unbalance_u │
│ 330034   ┆ 1408       ┆ 2020-10-11 11:43:45 

In [42]:
import datetime

datetime_list = [datetime.datetime(2020, 10, 6, 12, 11, 1), datetime.datetime(2020, 10, 30, 21, 57, 2), datetime.datetime(2020, 10, 31, 6, 35, 6), datetime.datetime(2020, 10, 31, 6, 38, 25), datetime.datetime(2020, 10, 31, 6, 38, 27), datetime.datetime(2020, 11, 2, 11, 36, 27), datetime.datetime(2020, 11, 5, 7, 45, 38), datetime.datetime(2020, 11, 5, 7, 47, 1), datetime.datetime(2020, 11, 7, 6, 14, 26), datetime.datetime(2020, 11, 7, 6, 14, 28), datetime.datetime(2020, 11, 26, 0, 59, 42), datetime.datetime(2021, 2, 21, 15, 49, 35), datetime.datetime(2021, 2, 21, 15, 58, 37), datetime.datetime(2021, 2, 21, 17, 36, 55), datetime.datetime(2021, 4, 24, 11, 34, 32), datetime.datetime(2021, 4, 24, 11, 35, 11), datetime.datetime(2021, 4, 24, 11, 38, 3), datetime.datetime(2021, 6, 22, 6, 2, 46), datetime.datetime(2021, 7, 30, 14, 57, 4), datetime.datetime(2021, 8, 14, 23, 44, 48), datetime.datetime(2021, 8, 16, 14, 37, 50), datetime.datetime(2021, 8, 29, 4, 18, 58), datetime.datetime(2021, 9, 29, 15, 51, 30), datetime.datetime(2021, 10, 14, 7, 24, 32), datetime.datetime(2021, 10, 26, 15, 11, 54), datetime.datetime(2021, 11, 19, 14, 18, 36), datetime.datetime(2022, 1, 5, 16, 54, 11), datetime.datetime(2022, 4, 13, 14, 44, 56), datetime.datetime(2022, 4, 19, 21, 28, 43), datetime.datetime(2022, 4, 20, 7, 34, 35), datetime.datetime(2022, 4, 21, 20, 31, 51), datetime.datetime(2022, 4, 22, 8, 1, 8), datetime.datetime(2022, 5, 2, 7, 40, 14), datetime.datetime(2022, 6, 9, 7, 10, 32), datetime.datetime(2022, 6, 9, 8, 38, 40), datetime.datetime(2022, 6, 9, 8, 39, 4), datetime.datetime(2022, 6, 28, 18, 58, 20), datetime.datetime(2022, 6, 28, 19, 51, 3), datetime.datetime(2022, 6, 28, 19, 55, 18), datetime.datetime(2022, 6, 28, 19, 56, 24), datetime.datetime(2022, 7, 13, 8, 28, 59), datetime.datetime(2022, 10, 13, 15, 51, 31), datetime.datetime(2022, 10, 31, 14, 23, 32), datetime.datetime(2022, 11, 3, 14, 54, 27), datetime.datetime(2022, 11, 7, 14, 54, 31), datetime.datetime(2023, 2, 19, 6, 26, 30), datetime.datetime(2023, 2, 19, 6, 59, 36), datetime.datetime(2023, 2, 19, 7, 0, 58), datetime.datetime(2023, 3, 23, 22, 29, 9), datetime.datetime(2023, 4, 13, 0, 12, 5), datetime.datetime(2023, 4, 13, 16, 42, 11), datetime.datetime(2023, 4, 15, 4, 44, 44), datetime.datetime(2023, 4, 15, 11, 36, 7), datetime.datetime(2023, 4, 16, 19, 39, 34), datetime.datetime(2023, 4, 29, 10, 31, 50), datetime.datetime(2023, 4, 30, 11, 39, 50), datetime.datetime(2023, 8, 6, 21, 41, 59), datetime.datetime(2023, 8, 8, 5, 50, 18), datetime.datetime(2023, 8, 8, 5, 52, 4), datetime.datetime(2023, 8, 8, 5, 58, 16), datetime.datetime(2023, 8, 8, 8, 40, 21), datetime.datetime(2023, 8, 8, 8, 57, 39), datetime.datetime(2023, 8, 8, 10, 50, 18), datetime.datetime(2023, 8, 8, 11, 30, 10), datetime.datetime(2023, 8, 15, 22, 8, 48), datetime.datetime(2023, 8, 22, 1, 24, 10), datetime.datetime(2023, 10, 7, 5, 44, 52), datetime.datetime(2023, 11, 26, 16, 44, 7), datetime.datetime(2023, 11, 27, 22, 11, 17), datetime.datetime(2023, 11, 27, 22, 12, 37), datetime.datetime(2023, 11, 27, 22, 16, 47), datetime.datetime(2023, 12, 3, 20, 34, 7), datetime.datetime(2023, 12, 3, 23, 20, 33), datetime.datetime(2023, 12, 3, 23, 22, 30), datetime.datetime(2023, 12, 3, 23, 33, 27), datetime.datetime(2023, 12, 13, 12, 11, 17), datetime.datetime(2023, 12, 13, 14, 32, 45), datetime.datetime(2023, 12, 13, 14, 35, 22), datetime.datetime(2023, 12, 16, 6, 49, 4), datetime.datetime(2023, 12, 22, 12, 16, 50), datetime.datetime(2023, 12, 22, 12, 18, 54), datetime.datetime(2023, 12, 22, 13, 32, 20), datetime.datetime(2024, 1, 7, 22, 42, 23), datetime.datetime(2024, 1, 7, 22, 43, 14), datetime.datetime(2024, 1, 7, 22, 51, 6), datetime.datetime(2024, 4, 2, 17, 27, 33), datetime.datetime(2024, 4, 5, 17, 17, 48), datetime.datetime(2024, 4, 9, 15, 45, 42), datetime.datetime(2024, 4, 16, 3, 5, 16), datetime.datetime(2024, 5, 2, 2, 37, 46), datetime.datetime(2024, 7, 23, 21, 33, 35), datetime.datetime(2024, 7, 23, 21, 35, 42), datetime.datetime(2024, 8, 30, 5, 51, 19), datetime.datetime(2024, 8, 30, 5, 53, 17), datetime.datetime(2024, 8, 30, 5, 55, 14), datetime.datetime(2024, 8, 31, 11, 27, 51), datetime.datetime(2024, 10, 8, 1, 34), datetime.datetime(2024, 10, 10, 6, 7, 58), datetime.datetime(2024, 10, 10, 13, 11, 26), datetime.datetime(2024, 10, 17, 13, 18, 42), datetime.datetime(2024, 10, 23, 11, 50, 9), datetime.datetime(2024, 10, 26, 12, 44, 31), datetime.datetime(2024, 11, 12, 9, 18, 51), datetime.datetime(2024, 12, 14, 18, 14, 49), datetime.datetime(2024, 12, 14, 18, 51, 34), datetime.datetime(2024, 12, 14, 19, 24, 22), datetime.datetime(2024, 12, 14, 19, 26, 15), datetime.datetime(2024, 12, 14, 20, 33, 18), datetime.datetime(2024, 12, 14, 22, 13, 54)]

timestamp_series = pl.Series(datetime_list, dtype=pl.Datetime(time_unit="us"))
timestamp_series


""
datetime[μs]
2020-10-06 12:11:01
2020-10-30 21:57:02
2020-10-31 06:35:06
2020-10-31 06:38:25
2020-10-31 06:38:27
2020-11-02 11:36:27
2020-11-05 07:45:38
2020-11-05 07:47:01
2020-11-07 06:14:26


In [12]:
interruption_df = pl.DataFrame({"timestamp": datetime_list})

# Function to calculate the time difference to the next interruption in minutes
def calculate_time_to_next_interruption(event_timestamp, interruption_df):
    # Find the first interruption that occurs after the current event timestamp
    future_interruption = interruption_df.filter(pl.col("timestamp") > event_timestamp)
    if future_interruption.height > 0:  # If there are interruptions after the event timestamp
        next_interruption = future_interruption[0, "timestamp"]  # Get the next interruption timestamp
        time_diff = next_interruption - event_timestamp  # Calculate the time difference
        return time_diff.total_seconds() / 60  # Return the time difference in minutes
    else:
        return None  # If no interruption is found, return None

# Now, assuming `events_df` contains your event data, here's how we modify it:
tte_df = (
    events_df.filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()  # Remove invalid rows
    )
    .sort(["subject_id", "timestamp"], descending=False)  # Sort by subject and timestamp
    .with_columns(
        pl.col("timestamp").apply(
            lambda x: calculate_time_to_next_interruption(x, interruption_df) if x else None
        ).alias("label")  # This will be the time to next interruption in minutes
    )
    .with_columns(
        pl.col("timestamp").min().over("subject_id").alias("start_time"),  # Get the minimum timestamp for the subject
        pl.col("timestamp").max().over("subject_id").alias("end_time")     # Get the maximum timestamp for the subject
    )
    .drop(["timestamp", "event_id", "event_type"])  # Drop the 'timestamp', 'event_id', and 'event_type' columns
)

tte_df.collect()

subject_id,label,start_time,end_time
u16,f64,datetime[μs],datetime[μs]
0,null,2025-02-01 13:51:48,2025-02-01 13:53:09
0,null,2025-02-01 13:51:48,2025-02-01 13:53:09
1,null,2025-02-01 13:51:48,2025-02-01 13:53:09
1,null,2025-02-01 13:51:48,2025-02-01 13:53:09
3,null,2025-01-24 07:02:57,2025-01-24 12:54:08
3,null,2025-01-24 07:02:57,2025-01-24 12:54:08
3,null,2025-01-24 07:02:57,2025-01-24 12:54:08
4,null,2025-01-23 23:10:07,2025-01-24 12:54:08
4,null,2025-01-23 23:10:07,2025-01-24 12:54:08


In [7]:
interruption_df.write_parquet(TASK_DF_DIR / "task_df_eneryield_interruption_cls.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/task_dfs


### Eneryield task df: binary classification if interruption is in the week after end_time, for each event sequence (subject)

In [15]:
COHORT_NAME = "eneryield_event_type"
PROJECT_DIR = Path('/home/filip-marcus/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/events_df.parquet...
shape: (330_052, 4)
┌──────────┬────────────┬─────────────────────┬─────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type  │
│ ---      ┆ ---        ┆ ---                 ┆ ---         │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat         │
╞══════════╪════════════╪═════════════════════╪═════════════╡
│ 5        ┆ 3          ┆ 2025-01-24 07:07:57 ┆ unbalance_i │
│ 14       ┆ 5          ┆ 2025-01-23 23:10:07 ┆ unbalance_i │
│ 19       ┆ 6          ┆ 2025-01-21 09:58:36 ┆ unbalance_i │
│ 23       ┆ 6          ┆ 2025-01-22 06:22:03 ┆ transient   │
│ …        ┆ …          ┆ …                   ┆ …           │
│ 330014   ┆ 1408       ┆ 2020-10-07 00:52:12 ┆ unbalance_u │
│ 330022   ┆ 1408       ┆ 2020-10-09 17:50:47 ┆ VD          │
│ 330029   ┆ 1408       ┆ 2020-10-11 11:32:57 ┆ unbalance_u │
│ 330034   ┆ 1408       ┆ 2020-10-11 11:43:45 ┆ unbal

In [16]:
import datetime

datetime_list = [datetime.datetime(2020, 10, 6, 12, 11, 1), datetime.datetime(2020, 10, 30, 21, 57, 2), datetime.datetime(2020, 10, 31, 6, 35, 6), datetime.datetime(2020, 10, 31, 6, 38, 25), datetime.datetime(2020, 10, 31, 6, 38, 27), datetime.datetime(2020, 11, 2, 11, 36, 27), datetime.datetime(2020, 11, 5, 7, 45, 38), datetime.datetime(2020, 11, 5, 7, 47, 1), datetime.datetime(2020, 11, 7, 6, 14, 26), datetime.datetime(2020, 11, 7, 6, 14, 28), datetime.datetime(2020, 11, 26, 0, 59, 42), datetime.datetime(2021, 2, 21, 15, 49, 35), datetime.datetime(2021, 2, 21, 15, 58, 37), datetime.datetime(2021, 2, 21, 17, 36, 55), datetime.datetime(2021, 4, 24, 11, 34, 32), datetime.datetime(2021, 4, 24, 11, 35, 11), datetime.datetime(2021, 4, 24, 11, 38, 3), datetime.datetime(2021, 6, 22, 6, 2, 46), datetime.datetime(2021, 7, 30, 14, 57, 4), datetime.datetime(2021, 8, 14, 23, 44, 48), datetime.datetime(2021, 8, 16, 14, 37, 50), datetime.datetime(2021, 8, 29, 4, 18, 58), datetime.datetime(2021, 9, 29, 15, 51, 30), datetime.datetime(2021, 10, 14, 7, 24, 32), datetime.datetime(2021, 10, 26, 15, 11, 54), datetime.datetime(2021, 11, 19, 14, 18, 36), datetime.datetime(2022, 1, 5, 16, 54, 11), datetime.datetime(2022, 4, 13, 14, 44, 56), datetime.datetime(2022, 4, 19, 21, 28, 43), datetime.datetime(2022, 4, 20, 7, 34, 35), datetime.datetime(2022, 4, 21, 20, 31, 51), datetime.datetime(2022, 4, 22, 8, 1, 8), datetime.datetime(2022, 5, 2, 7, 40, 14), datetime.datetime(2022, 6, 9, 7, 10, 32), datetime.datetime(2022, 6, 9, 8, 38, 40), datetime.datetime(2022, 6, 9, 8, 39, 4), datetime.datetime(2022, 6, 28, 18, 58, 20), datetime.datetime(2022, 6, 28, 19, 51, 3), datetime.datetime(2022, 6, 28, 19, 55, 18), datetime.datetime(2022, 6, 28, 19, 56, 24), datetime.datetime(2022, 7, 13, 8, 28, 59), datetime.datetime(2022, 10, 13, 15, 51, 31), datetime.datetime(2022, 10, 31, 14, 23, 32), datetime.datetime(2022, 11, 3, 14, 54, 27), datetime.datetime(2022, 11, 7, 14, 54, 31), datetime.datetime(2023, 2, 19, 6, 26, 30), datetime.datetime(2023, 2, 19, 6, 59, 36), datetime.datetime(2023, 2, 19, 7, 0, 58), datetime.datetime(2023, 3, 23, 22, 29, 9), datetime.datetime(2023, 4, 13, 0, 12, 5), datetime.datetime(2023, 4, 13, 16, 42, 11), datetime.datetime(2023, 4, 15, 4, 44, 44), datetime.datetime(2023, 4, 15, 11, 36, 7), datetime.datetime(2023, 4, 16, 19, 39, 34), datetime.datetime(2023, 4, 29, 10, 31, 50), datetime.datetime(2023, 4, 30, 11, 39, 50), datetime.datetime(2023, 8, 6, 21, 41, 59), datetime.datetime(2023, 8, 8, 5, 50, 18), datetime.datetime(2023, 8, 8, 5, 52, 4), datetime.datetime(2023, 8, 8, 5, 58, 16), datetime.datetime(2023, 8, 8, 8, 40, 21), datetime.datetime(2023, 8, 8, 8, 57, 39), datetime.datetime(2023, 8, 8, 10, 50, 18), datetime.datetime(2023, 8, 8, 11, 30, 10), datetime.datetime(2023, 8, 15, 22, 8, 48), datetime.datetime(2023, 8, 22, 1, 24, 10), datetime.datetime(2023, 10, 7, 5, 44, 52), datetime.datetime(2023, 11, 26, 16, 44, 7), datetime.datetime(2023, 11, 27, 22, 11, 17), datetime.datetime(2023, 11, 27, 22, 12, 37), datetime.datetime(2023, 11, 27, 22, 16, 47), datetime.datetime(2023, 12, 3, 20, 34, 7), datetime.datetime(2023, 12, 3, 23, 20, 33), datetime.datetime(2023, 12, 3, 23, 22, 30), datetime.datetime(2023, 12, 3, 23, 33, 27), datetime.datetime(2023, 12, 13, 12, 11, 17), datetime.datetime(2023, 12, 13, 14, 32, 45), datetime.datetime(2023, 12, 13, 14, 35, 22), datetime.datetime(2023, 12, 16, 6, 49, 4), datetime.datetime(2023, 12, 22, 12, 16, 50), datetime.datetime(2023, 12, 22, 12, 18, 54), datetime.datetime(2023, 12, 22, 13, 32, 20), datetime.datetime(2024, 1, 7, 22, 42, 23), datetime.datetime(2024, 1, 7, 22, 43, 14), datetime.datetime(2024, 1, 7, 22, 51, 6), datetime.datetime(2024, 4, 2, 17, 27, 33), datetime.datetime(2024, 4, 5, 17, 17, 48), datetime.datetime(2024, 4, 9, 15, 45, 42), datetime.datetime(2024, 4, 16, 3, 5, 16), datetime.datetime(2024, 5, 2, 2, 37, 46), datetime.datetime(2024, 7, 23, 21, 33, 35), datetime.datetime(2024, 7, 23, 21, 35, 42), datetime.datetime(2024, 8, 30, 5, 51, 19), datetime.datetime(2024, 8, 30, 5, 53, 17), datetime.datetime(2024, 8, 30, 5, 55, 14), datetime.datetime(2024, 8, 31, 11, 27, 51), datetime.datetime(2024, 10, 8, 1, 34), datetime.datetime(2024, 10, 10, 6, 7, 58), datetime.datetime(2024, 10, 10, 13, 11, 26), datetime.datetime(2024, 10, 17, 13, 18, 42), datetime.datetime(2024, 10, 23, 11, 50, 9), datetime.datetime(2024, 10, 26, 12, 44, 31), datetime.datetime(2024, 11, 12, 9, 18, 51), datetime.datetime(2024, 12, 14, 18, 14, 49), datetime.datetime(2024, 12, 14, 18, 51, 34), datetime.datetime(2024, 12, 14, 19, 24, 22), datetime.datetime(2024, 12, 14, 19, 26, 15), datetime.datetime(2024, 12, 14, 20, 33, 18), datetime.datetime(2024, 12, 14, 22, 13, 54)]

timestamp_series = pl.Series(datetime_list, dtype=pl.Datetime(time_unit="us"))
timestamp_series

""
datetime[μs]
2020-10-06 12:11:01
2020-10-30 21:57:02
2020-10-31 06:35:06
2020-10-31 06:38:25
2020-10-31 06:38:27
2020-11-02 11:36:27
2020-11-05 07:45:38
2020-11-05 07:47:01
2020-11-07 06:14:26


In [17]:
from datetime import datetime, timedelta
aggregated_df = (
        events_df.filter(
            pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()
        )
        .sort(["subject_id", "timestamp"], descending=False)
        .groupby("subject_id")
        .agg(
            pl.col("timestamp").min().alias("start_time"),
            pl.col("timestamp").max().alias("end_time"),
        )
    ).collect()

interruption_df = aggregated_df.with_columns(
        pl.Series([
            int(any(end <= ts <= end + timedelta(weeks=1) for ts in timestamp_series))
            for end in aggregated_df["end_time"].to_list()
        ]).alias("label")
    )
interruption_df

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],i64
0,2025-02-01 13:51:48,2025-02-01 13:53:09,0
1,2025-02-01 13:51:48,2025-02-01 13:53:09,0
3,2025-01-24 07:02:57,2025-01-24 12:54:08,0
4,2025-01-23 23:10:07,2025-01-24 12:54:08,0
5,2025-01-22 06:22:03,2025-01-24 12:54:08,0
6,2025-01-21 09:58:36,2025-01-24 12:54:08,0
7,2025-01-20 04:26:17,2025-01-24 12:54:08,0
8,2025-01-20 04:26:17,2025-01-24 12:54:08,0
9,2025-01-20 04:26:17,2025-01-24 12:54:08,0


In [18]:
interruption_df_copy = interruption_df
interruption_df_copy = interruption_df_copy.with_columns(
    pl.Series("label", tti_labels)
)
interruption_df_copy

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],f64
0,2025-02-01 13:51:48,2025-02-01 13:53:09,0.0
1,2025-02-01 13:51:48,2025-02-01 13:53:09,0.0
3,2025-01-24 07:02:57,2025-01-24 12:54:08,0.0
4,2025-01-23 23:10:07,2025-01-24 12:54:08,0.0
5,2025-01-22 06:22:03,2025-01-24 12:54:08,0.0
6,2025-01-21 09:58:36,2025-01-24 12:54:08,0.0
7,2025-01-20 04:26:17,2025-01-24 12:54:08,0.0
8,2025-01-20 04:26:17,2025-01-24 12:54:08,0.0
9,2025-01-20 04:26:17,2025-01-24 12:54:08,0.0


In [19]:
interruption_df_copy.write_parquet(TASK_DF_DIR / "task_df_eneryield_tti")

In [28]:
interruption_df.write_parquet(TASK_DF_DIR / "task_df_eneryield_interruption_cls_one_week_ahead.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/task_dfs


### Time-to-Interruption

In [47]:
COHORT_NAME = "synthetic_mega"
PROJECT_DIR = Path('/home/filip-marcus/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/synthetic_mega/events_df.parquet...
shape: (209_601, 4)
┌──────────┬────────────┬─────────────────────┬─────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type  │
│ ---      ┆ ---        ┆ ---                 ┆ ---         │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat         │
╞══════════╪════════════╪═════════════════════╪═════════════╡
│ 9        ┆ 0          ┆ 2024-12-26 16:49:41 ┆ disturbance │
│ 10       ┆ 0          ┆ 2024-12-26 21:12:47 ┆ disturbance │
│ 14       ┆ 0          ┆ 2024-12-27 14:45:15 ┆ disturbance │
│ 15       ┆ 0          ┆ 2024-12-27 19:08:22 ┆ disturbance │
│ …        ┆ …          ┆ …                   ┆ …           │
│ 209578   ┆ 5502       ┆ 2020-01-01 12:51:21 ┆ disturbance │
│ 209588   ┆ 5503       ┆ 2020-01-01 00:00:00 ┆ disturbance │
│ 209591   ┆ 5503       ┆ 2020-01-01 06:44:09 ┆ disturbance │
│ 209593   ┆ 5503       ┆ 2020-01-01 09:55:18 ┆ disturbance

In [48]:
import datetime

datetime_list = [datetime.datetime(2020, 10, 6, 12, 11, 1), datetime.datetime(2020, 10, 30, 21, 57, 2), datetime.datetime(2020, 10, 31, 6, 35, 6), datetime.datetime(2020, 10, 31, 6, 38, 25), datetime.datetime(2020, 10, 31, 6, 38, 27), datetime.datetime(2020, 11, 2, 11, 36, 27), datetime.datetime(2020, 11, 5, 7, 45, 38), datetime.datetime(2020, 11, 5, 7, 47, 1), datetime.datetime(2020, 11, 7, 6, 14, 26), datetime.datetime(2020, 11, 7, 6, 14, 28), datetime.datetime(2020, 11, 26, 0, 59, 42), datetime.datetime(2021, 2, 21, 15, 49, 35), datetime.datetime(2021, 2, 21, 15, 58, 37), datetime.datetime(2021, 2, 21, 17, 36, 55), datetime.datetime(2021, 4, 24, 11, 34, 32), datetime.datetime(2021, 4, 24, 11, 35, 11), datetime.datetime(2021, 4, 24, 11, 38, 3), datetime.datetime(2021, 6, 22, 6, 2, 46), datetime.datetime(2021, 7, 30, 14, 57, 4), datetime.datetime(2021, 8, 14, 23, 44, 48), datetime.datetime(2021, 8, 16, 14, 37, 50), datetime.datetime(2021, 8, 29, 4, 18, 58), datetime.datetime(2021, 9, 29, 15, 51, 30), datetime.datetime(2021, 10, 14, 7, 24, 32), datetime.datetime(2021, 10, 26, 15, 11, 54), datetime.datetime(2021, 11, 19, 14, 18, 36), datetime.datetime(2022, 1, 5, 16, 54, 11), datetime.datetime(2022, 4, 13, 14, 44, 56), datetime.datetime(2022, 4, 19, 21, 28, 43), datetime.datetime(2022, 4, 20, 7, 34, 35), datetime.datetime(2022, 4, 21, 20, 31, 51), datetime.datetime(2022, 4, 22, 8, 1, 8), datetime.datetime(2022, 5, 2, 7, 40, 14), datetime.datetime(2022, 6, 9, 7, 10, 32), datetime.datetime(2022, 6, 9, 8, 38, 40), datetime.datetime(2022, 6, 9, 8, 39, 4), datetime.datetime(2022, 6, 28, 18, 58, 20), datetime.datetime(2022, 6, 28, 19, 51, 3), datetime.datetime(2022, 6, 28, 19, 55, 18), datetime.datetime(2022, 6, 28, 19, 56, 24), datetime.datetime(2022, 7, 13, 8, 28, 59), datetime.datetime(2022, 10, 13, 15, 51, 31), datetime.datetime(2022, 10, 31, 14, 23, 32), datetime.datetime(2022, 11, 3, 14, 54, 27), datetime.datetime(2022, 11, 7, 14, 54, 31), datetime.datetime(2023, 2, 19, 6, 26, 30), datetime.datetime(2023, 2, 19, 6, 59, 36), datetime.datetime(2023, 2, 19, 7, 0, 58), datetime.datetime(2023, 3, 23, 22, 29, 9), datetime.datetime(2023, 4, 13, 0, 12, 5), datetime.datetime(2023, 4, 13, 16, 42, 11), datetime.datetime(2023, 4, 15, 4, 44, 44), datetime.datetime(2023, 4, 15, 11, 36, 7), datetime.datetime(2023, 4, 16, 19, 39, 34), datetime.datetime(2023, 4, 29, 10, 31, 50), datetime.datetime(2023, 4, 30, 11, 39, 50), datetime.datetime(2023, 8, 6, 21, 41, 59), datetime.datetime(2023, 8, 8, 5, 50, 18), datetime.datetime(2023, 8, 8, 5, 52, 4), datetime.datetime(2023, 8, 8, 5, 58, 16), datetime.datetime(2023, 8, 8, 8, 40, 21), datetime.datetime(2023, 8, 8, 8, 57, 39), datetime.datetime(2023, 8, 8, 10, 50, 18), datetime.datetime(2023, 8, 8, 11, 30, 10), datetime.datetime(2023, 8, 15, 22, 8, 48), datetime.datetime(2023, 8, 22, 1, 24, 10), datetime.datetime(2023, 10, 7, 5, 44, 52), datetime.datetime(2023, 11, 26, 16, 44, 7), datetime.datetime(2023, 11, 27, 22, 11, 17), datetime.datetime(2023, 11, 27, 22, 12, 37), datetime.datetime(2023, 11, 27, 22, 16, 47), datetime.datetime(2023, 12, 3, 20, 34, 7), datetime.datetime(2023, 12, 3, 23, 20, 33), datetime.datetime(2023, 12, 3, 23, 22, 30), datetime.datetime(2023, 12, 3, 23, 33, 27), datetime.datetime(2023, 12, 13, 12, 11, 17), datetime.datetime(2023, 12, 13, 14, 32, 45), datetime.datetime(2023, 12, 13, 14, 35, 22), datetime.datetime(2023, 12, 16, 6, 49, 4), datetime.datetime(2023, 12, 22, 12, 16, 50), datetime.datetime(2023, 12, 22, 12, 18, 54), datetime.datetime(2023, 12, 22, 13, 32, 20), datetime.datetime(2024, 1, 7, 22, 42, 23), datetime.datetime(2024, 1, 7, 22, 43, 14), datetime.datetime(2024, 1, 7, 22, 51, 6), datetime.datetime(2024, 4, 2, 17, 27, 33), datetime.datetime(2024, 4, 5, 17, 17, 48), datetime.datetime(2024, 4, 9, 15, 45, 42), datetime.datetime(2024, 4, 16, 3, 5, 16), datetime.datetime(2024, 5, 2, 2, 37, 46), datetime.datetime(2024, 7, 23, 21, 33, 35), datetime.datetime(2024, 7, 23, 21, 35, 42), datetime.datetime(2024, 8, 30, 5, 51, 19), datetime.datetime(2024, 8, 30, 5, 53, 17), datetime.datetime(2024, 8, 30, 5, 55, 14), datetime.datetime(2024, 8, 31, 11, 27, 51), datetime.datetime(2024, 10, 8, 1, 34), datetime.datetime(2024, 10, 10, 6, 7, 58), datetime.datetime(2024, 10, 10, 13, 11, 26), datetime.datetime(2024, 10, 17, 13, 18, 42), datetime.datetime(2024, 10, 23, 11, 50, 9), datetime.datetime(2024, 10, 26, 12, 44, 31), datetime.datetime(2024, 11, 12, 9, 18, 51), datetime.datetime(2024, 12, 14, 18, 14, 49), datetime.datetime(2024, 12, 14, 18, 51, 34), datetime.datetime(2024, 12, 14, 19, 24, 22), datetime.datetime(2024, 12, 14, 19, 26, 15), datetime.datetime(2024, 12, 14, 20, 33, 18), datetime.datetime(2024, 12, 14, 22, 13, 54)]
 
timestamp_series = pl.Series(datetime_list, dtype=pl.Datetime(time_unit="us"))
timestamp_series

""
datetime[μs]
2020-10-06 12:11:01
2020-10-30 21:57:02
2020-10-31 06:35:06
2020-10-31 06:38:25
2020-10-31 06:38:27
2020-11-02 11:36:27
2020-11-05 07:45:38
2020-11-05 07:47:01
2020-11-07 06:14:26


In [30]:
interruption_in_seq_df = pl.read_parquet('/home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/task_dfs/task_df_eneryield_interruption_cls.parquet')
tti_df2 = interruption_in_seq_df.with_columns(pl.lit(1).alias("label"))
tti_df2

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],i32
0,2025-02-01 13:51:48,2025-02-01 13:53:09,1
1,2025-02-01 13:51:48,2025-02-01 13:53:09,1
3,2025-01-24 07:02:57,2025-01-24 12:54:08,1
4,2025-01-23 23:10:07,2025-01-24 12:54:08,1
5,2025-01-22 06:22:03,2025-01-24 12:54:08,1
6,2025-01-21 09:58:36,2025-01-24 12:54:08,1
7,2025-01-20 04:26:17,2025-01-24 12:54:08,1
8,2025-01-20 04:26:17,2025-01-24 12:54:08,1
9,2025-01-20 04:26:17,2025-01-24 12:54:08,1


In [49]:
from bisect import bisect_right

# Ensure datetime_list is sorted.
datetime_list.sort()

# Group events_df by subject_id to get start_time (min) and end_time (max)
subject_df = (
    events_df
    .groupby("subject_id")
    .agg([
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    ])
).collect()

# Define a function that, given an end_time, computes time-to-next interruption (in minutes)
def compute_time_to_next_interrupt(end_time):
    # bisect_right returns the index where end_time would be inserted to keep datetime_list sorted.
    idx = bisect_right(datetime_list, end_time)
    if idx < len(datetime_list):
        next_interrupt = datetime_list[idx]
        # Compute difference in minutes
        diff_minutes = (next_interrupt - end_time).total_seconds() / 60.0
        return diff_minutes
    else:
        # Return None if no interruption occurs after end_time
        return None

# Apply the function row-wise to create the "label" column.
tti_df = subject_df.with_columns(
    pl.col("end_time").apply(compute_time_to_next_interrupt).alias("label")
)
tti_df = tti_df.sort(by="subject_id")
tti_df = tti_df.drop_nulls(subset=["label"])

In [50]:
tti_labels = tti_df["label"]
tti_labels

label
f64
46.05
1223.15
2538.716667
4117.4
5432.966667
7011.65
8327.233333
9905.916667
11221.483333


In [51]:
# TA BORT GAMLA INNAN!!!!
tti_df.write_parquet(TASK_DF_DIR / "task_df_synthetic_tti.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/synthetic_mega/task_dfs


### Eneryield Class dist

In [18]:
COHORT_NAME = "eneryield_event_type"
PROJECT_DIR = Path('/home/filip-marcus/models/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False)

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/events_df.parquet...
shape: (330_052, 4)
┌──────────┬────────────┬─────────────────────┬─────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type  │
│ ---      ┆ ---        ┆ ---                 ┆ ---         │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat         │
╞══════════╪════════════╪═════════════════════╪═════════════╡
│ 5        ┆ 3          ┆ 2025-01-24 07:07:57 ┆ unbalance_i │
│ 14       ┆ 5          ┆ 2025-01-23 23:10:07 ┆ unbalance_i │
│ 19       ┆ 6          ┆ 2025-01-21 09:58:36 ┆ unbalance_i │
│ 23       ┆ 6          ┆ 2025-01-22 06:22:03 ┆ transient   │
│ …        ┆ …          ┆ …                   ┆ …           │
│ 330014   ┆ 1408       ┆ 2020-10-07 00:52:12 ┆ unbalance_u │
│ 330022   ┆ 1408       ┆ 2020-10-09 17:50:47 ┆ VD          │
│ 330029   ┆ 1408       ┆ 2020-10-11 11:32:57 ┆ unbalance_u │
│ 330034   ┆ 1408       ┆ 2020-10-11 11:43:45 

In [19]:
import datetime

datetime_list = [datetime.datetime(2020, 10, 6, 12, 11, 1), datetime.datetime(2020, 10, 30, 21, 57, 2), datetime.datetime(2020, 10, 31, 6, 35, 6), datetime.datetime(2020, 10, 31, 6, 38, 25), datetime.datetime(2020, 10, 31, 6, 38, 27), datetime.datetime(2020, 11, 2, 11, 36, 27), datetime.datetime(2020, 11, 5, 7, 45, 38), datetime.datetime(2020, 11, 5, 7, 47, 1), datetime.datetime(2020, 11, 7, 6, 14, 26), datetime.datetime(2020, 11, 7, 6, 14, 28), datetime.datetime(2020, 11, 26, 0, 59, 42), datetime.datetime(2021, 2, 21, 15, 49, 35), datetime.datetime(2021, 2, 21, 15, 58, 37), datetime.datetime(2021, 2, 21, 17, 36, 55), datetime.datetime(2021, 4, 24, 11, 34, 32), datetime.datetime(2021, 4, 24, 11, 35, 11), datetime.datetime(2021, 4, 24, 11, 38, 3), datetime.datetime(2021, 6, 22, 6, 2, 46), datetime.datetime(2021, 7, 30, 14, 57, 4), datetime.datetime(2021, 8, 14, 23, 44, 48), datetime.datetime(2021, 8, 16, 14, 37, 50), datetime.datetime(2021, 8, 29, 4, 18, 58), datetime.datetime(2021, 9, 29, 15, 51, 30), datetime.datetime(2021, 10, 14, 7, 24, 32), datetime.datetime(2021, 10, 26, 15, 11, 54), datetime.datetime(2021, 11, 19, 14, 18, 36), datetime.datetime(2022, 1, 5, 16, 54, 11), datetime.datetime(2022, 4, 13, 14, 44, 56), datetime.datetime(2022, 4, 19, 21, 28, 43), datetime.datetime(2022, 4, 20, 7, 34, 35), datetime.datetime(2022, 4, 21, 20, 31, 51), datetime.datetime(2022, 4, 22, 8, 1, 8), datetime.datetime(2022, 5, 2, 7, 40, 14), datetime.datetime(2022, 6, 9, 7, 10, 32), datetime.datetime(2022, 6, 9, 8, 38, 40), datetime.datetime(2022, 6, 9, 8, 39, 4), datetime.datetime(2022, 6, 28, 18, 58, 20), datetime.datetime(2022, 6, 28, 19, 51, 3), datetime.datetime(2022, 6, 28, 19, 55, 18), datetime.datetime(2022, 6, 28, 19, 56, 24), datetime.datetime(2022, 7, 13, 8, 28, 59), datetime.datetime(2022, 10, 13, 15, 51, 31), datetime.datetime(2022, 10, 31, 14, 23, 32), datetime.datetime(2022, 11, 3, 14, 54, 27), datetime.datetime(2022, 11, 7, 14, 54, 31), datetime.datetime(2023, 2, 19, 6, 26, 30), datetime.datetime(2023, 2, 19, 6, 59, 36), datetime.datetime(2023, 2, 19, 7, 0, 58), datetime.datetime(2023, 3, 23, 22, 29, 9), datetime.datetime(2023, 4, 13, 0, 12, 5), datetime.datetime(2023, 4, 13, 16, 42, 11), datetime.datetime(2023, 4, 15, 4, 44, 44), datetime.datetime(2023, 4, 15, 11, 36, 7), datetime.datetime(2023, 4, 16, 19, 39, 34), datetime.datetime(2023, 4, 29, 10, 31, 50), datetime.datetime(2023, 4, 30, 11, 39, 50), datetime.datetime(2023, 8, 6, 21, 41, 59), datetime.datetime(2023, 8, 8, 5, 50, 18), datetime.datetime(2023, 8, 8, 5, 52, 4), datetime.datetime(2023, 8, 8, 5, 58, 16), datetime.datetime(2023, 8, 8, 8, 40, 21), datetime.datetime(2023, 8, 8, 8, 57, 39), datetime.datetime(2023, 8, 8, 10, 50, 18), datetime.datetime(2023, 8, 8, 11, 30, 10), datetime.datetime(2023, 8, 15, 22, 8, 48), datetime.datetime(2023, 8, 22, 1, 24, 10), datetime.datetime(2023, 10, 7, 5, 44, 52), datetime.datetime(2023, 11, 26, 16, 44, 7), datetime.datetime(2023, 11, 27, 22, 11, 17), datetime.datetime(2023, 11, 27, 22, 12, 37), datetime.datetime(2023, 11, 27, 22, 16, 47), datetime.datetime(2023, 12, 3, 20, 34, 7), datetime.datetime(2023, 12, 3, 23, 20, 33), datetime.datetime(2023, 12, 3, 23, 22, 30), datetime.datetime(2023, 12, 3, 23, 33, 27), datetime.datetime(2023, 12, 13, 12, 11, 17), datetime.datetime(2023, 12, 13, 14, 32, 45), datetime.datetime(2023, 12, 13, 14, 35, 22), datetime.datetime(2023, 12, 16, 6, 49, 4), datetime.datetime(2023, 12, 22, 12, 16, 50), datetime.datetime(2023, 12, 22, 12, 18, 54), datetime.datetime(2023, 12, 22, 13, 32, 20), datetime.datetime(2024, 1, 7, 22, 42, 23), datetime.datetime(2024, 1, 7, 22, 43, 14), datetime.datetime(2024, 1, 7, 22, 51, 6), datetime.datetime(2024, 4, 2, 17, 27, 33), datetime.datetime(2024, 4, 5, 17, 17, 48), datetime.datetime(2024, 4, 9, 15, 45, 42), datetime.datetime(2024, 4, 16, 3, 5, 16), datetime.datetime(2024, 5, 2, 2, 37, 46), datetime.datetime(2024, 7, 23, 21, 33, 35), datetime.datetime(2024, 7, 23, 21, 35, 42), datetime.datetime(2024, 8, 30, 5, 51, 19), datetime.datetime(2024, 8, 30, 5, 53, 17), datetime.datetime(2024, 8, 30, 5, 55, 14), datetime.datetime(2024, 8, 31, 11, 27, 51), datetime.datetime(2024, 10, 8, 1, 34), datetime.datetime(2024, 10, 10, 6, 7, 58), datetime.datetime(2024, 10, 10, 13, 11, 26), datetime.datetime(2024, 10, 17, 13, 18, 42), datetime.datetime(2024, 10, 23, 11, 50, 9), datetime.datetime(2024, 10, 26, 12, 44, 31), datetime.datetime(2024, 11, 12, 9, 18, 51), datetime.datetime(2024, 12, 14, 18, 14, 49), datetime.datetime(2024, 12, 14, 18, 51, 34), datetime.datetime(2024, 12, 14, 19, 24, 22), datetime.datetime(2024, 12, 14, 19, 26, 15), datetime.datetime(2024, 12, 14, 20, 33, 18), datetime.datetime(2024, 12, 14, 22, 13, 54)]

timestamp_series = pl.Series(datetime_list, dtype=pl.Datetime(time_unit="us"))
timestamp_series

""
datetime[μs]
2020-10-06 12:11:01
2020-10-30 21:57:02
2020-10-31 06:35:06
2020-10-31 06:38:25
2020-10-31 06:38:27
2020-11-02 11:36:27
2020-11-05 07:45:38
2020-11-05 07:47:01
2020-11-07 06:14:26


In [20]:
from datetime import datetime, timedelta
aggregated_df = (
        events_df.filter(
            pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()
        )
        .sort(["subject_id", "timestamp"], descending=False)
        .groupby("subject_id")
        .agg(
            pl.col("timestamp").min().alias("start_time"),
            pl.col("timestamp").max().alias("end_time"),
        )
    ).collect()

interruption_df = aggregated_df.with_columns(
        pl.Series([
            int(any(end <= ts <= end + timedelta(weeks=1) for ts in timestamp_series))
            for end in aggregated_df["end_time"].to_list()
        ]).alias("label")
    )
interruption_df

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],i64
0,2025-02-01 13:51:48,2025-02-01 13:53:09,0
1,2025-02-01 13:51:48,2025-02-01 13:53:09,0
3,2025-01-24 07:02:57,2025-01-24 12:54:08,0
4,2025-01-23 23:10:07,2025-01-24 12:54:08,0
5,2025-01-22 06:22:03,2025-01-24 12:54:08,0
6,2025-01-21 09:58:36,2025-01-24 12:54:08,0
7,2025-01-20 04:26:17,2025-01-24 12:54:08,0
8,2025-01-20 04:26:17,2025-01-24 12:54:08,0
9,2025-01-20 04:26:17,2025-01-24 12:54:08,0


In [21]:
# labels = "[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"

# # Create a column where every row has the same label string
# label_column = pl.Series("label", [labels] * len(interruption_df))

# # Add the new column to the DataFrame
# interruption_df = interruption_df.with_columns(label_column)



labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Create a column where every row has the same list
label_column = pl.Series("label", [labels] * len(interruption_df))

# Add the new column to the DataFrame
interruption_df = interruption_df.with_columns(label_column)



interruption_df

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],list[i64]
0,2025-02-01 13:51:48,2025-02-01 13:53:09,"[0, 1, … 10]"
1,2025-02-01 13:51:48,2025-02-01 13:53:09,"[0, 1, … 10]"
3,2025-01-24 07:02:57,2025-01-24 12:54:08,"[0, 1, … 10]"
4,2025-01-23 23:10:07,2025-01-24 12:54:08,"[0, 1, … 10]"
5,2025-01-22 06:22:03,2025-01-24 12:54:08,"[0, 1, … 10]"
6,2025-01-21 09:58:36,2025-01-24 12:54:08,"[0, 1, … 10]"
7,2025-01-20 04:26:17,2025-01-24 12:54:08,"[0, 1, … 10]"
8,2025-01-20 04:26:17,2025-01-24 12:54:08,"[0, 1, … 10]"
9,2025-01-20 04:26:17,2025-01-24 12:54:08,"[0, 1, … 10]"


In [23]:
interruption_df.write_parquet(TASK_DF_DIR / "task_df_eneryield_class_dist_test.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/models/ESGPT_new/EventStreamGPT/data/processed/eneryield_event_type/task_dfs


### BUILD TASK DFS, 2025-04-08

In [20]:
COHORT_NAME = "giga_mind_final"
PROJECT_DIR = Path('/home/filip-marcus/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False) 

ESD = Dataset.load(DATA_DIR)

# ORIGINAL - ENERYIELD
# events_df = ESD.events_df.lazy()


# FOR FOUNDATION MODEL ONLY!
events_df = (
    ESD.events_df.lazy()
    .filter(pl.col("event_type") == "eneryield_event")
)

print(events_df.collect().unique())

Loading events from /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/giga_mind_final/events_df.parquet...
shape: (78_010, 4)
┌──────────┬────────────┬─────────────────────┬─────────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type      │
│ ---      ┆ ---        ┆ ---                 ┆ ---             │
│ u32      ┆ u32        ┆ datetime[μs]        ┆ cat             │
╞══════════╪════════════╪═════════════════════╪═════════════════╡
│ 5026838  ┆ 201171     ┆ 2020-10-06 12:11:01 ┆ eneryield_event │
│ 5026843  ┆ 201171     ┆ 2020-10-06 13:18:33 ┆ eneryield_event │
│ 5026847  ┆ 201172     ┆ 2020-10-08 07:48:21 ┆ eneryield_event │
│ 5026859  ┆ 201173     ┆ 2020-10-11 11:32:57 ┆ eneryield_event │
│ …        ┆ …          ┆ …                   ┆ …               │
│ 5104821  ┆ 201615     ┆ 2025-02-03 09:24:02 ┆ eneryield_event │
│ 5104829  ┆ 201615     ┆ 2025-02-03 10:05:43 ┆ eneryield_event │
│ 5104836  ┆ 201615     ┆ 2025-02-03 11:14:22 ┆ eneryield_event │
│ 51048

In [21]:
import pandas as pd
dataset_0 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_0.csv')#test
dataset_1 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_1.csv')
dataset_2 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_2.csv')
dataset_3 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_3.csv')
dataset_4 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_4.csv')
dataset_5 = pd.read_csv('/home/filip-marcus/preprocessed_data/eneryield/pretrain/eneryield_dataset_1.csv')
dataset_6 = pd.read_csv('/home/filip-marcus/preprocessed_data/eneryield/pretrain/eneryield_dataset_2.csv')
dataset_7 = pd.read_csv('/home/filip-marcus/preprocessed_data/gigamind/pretrain/eneryield.csv')

In [22]:
# Filter the dataset_1 to get rows where event_label is 'interruption'
subjects_with_interruption = dataset_7[dataset_7['event_label'] == 'interruption']['sequence_id'].unique().tolist()
subjects_with_interruption

[201171,
 201179,
 201180,
 201181,
 201182,
 201188,
 201201,
 201212,
 201222,
 201227,
 201232,
 201233,
 201237,
 201247,
 201252,
 201255,
 201263,
 201278,
 201304,
 201306,
 201307,
 201310,
 201319,
 201322,
 201327,
 201343,
 201349,
 201350,
 201351,
 201386,
 201396,
 201403,
 201404,
 201409,
 201440,
 201443,
 201445,
 201460,
 201474,
 201476,
 201480,
 201481,
 201483,
 201488,
 201517,
 201518,
 201519,
 201521,
 201527,
 201554,
 201566,
 201579,
 201580,
 201582,
 201583,
 201587,
 201598]

In [18]:
# INTERRUPTION IN SEQUENCE
aggregated_df = (
    events_df.filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()
    )
    .sort(["subject_id", "timestamp"])  # Sort by subject_id and timestamp
    .with_columns([
        # Ensure the 'timestamp' column is correctly cast to datetime[μs] if it's not already
        pl.col("timestamp").cast(pl.Datetime(time_unit="us")).alias("timestamp")
    ])
    .groupby("subject_id")  # Group by subject_id
    .agg([
        pl.col("timestamp").min().alias("start_time"),  # Get the min timestamp as start_time
        pl.col("timestamp").max().alias("end_time"),    # Get the max timestamp as end_time
    ])
    .collect()  # Collect the result into a DataFrame
)
task_df = aggregated_df.with_columns(
    pl.col("subject_id").is_in(subjects_with_interruption).cast(pl.Int32).alias("label")
)

task_df

subject_id,start_time,end_time,label
u32,datetime[μs],datetime[μs],i32
201171,2020-10-06 12:11:01,2020-10-07 05:41:14,1
201172,2020-10-08 07:48:21,2020-10-10 13:16:56,0
201173,2020-10-11 11:32:57,2020-10-13 14:55:12,0
201174,2020-10-14 05:11:08,2020-10-16 14:37:09,0
201175,2020-10-17 05:38:55,2020-10-19 10:46:58,0
201176,2020-10-20 06:00:03,2020-10-22 13:35:38,0
201177,2020-10-23 09:56:49,2020-10-25 14:11:15,0
201178,2020-10-26 11:10:38,2020-10-28 23:09:48,0
201179,2020-10-29 06:55:57,2020-10-31 06:39:27,1


In [30]:
# INTERRUPTION NEXT WEEK BY SHIFTING LABELS
task_shifted_df = task_df.with_columns(task_df["label"].shift(-1).alias("label")).fill_null(0)
task_shifted_df

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],i32
0,2020-10-06 12:11:01,2020-10-07 05:41:14,0
1,2020-10-08 07:48:21,2020-10-10 13:16:56,0
2,2020-10-11 11:32:57,2020-10-13 14:55:12,0
3,2020-10-14 05:11:08,2020-10-16 14:37:09,0
4,2020-10-17 05:38:55,2020-10-19 10:46:58,0
5,2020-10-20 06:00:03,2020-10-22 13:35:38,0
6,2020-10-23 09:56:49,2020-10-25 14:11:15,0
7,2020-10-26 11:10:38,2020-10-28 23:09:48,1
8,2020-10-29 06:55:57,2020-10-31 06:39:27,1


In [27]:
# INTERRUPTION IN NEXT K DAYS
prediction_window_days = 7 # number of days to look forward

from datetime import timedelta
interruption_df = pl.read_csv('/home/filip-marcus/preprocessed_data/eneryield/interruption_times.csv')
interruption_df = interruption_df.with_columns(
    pl.col("interruption_time").str.strptime(pl.Datetime("us"), format="%Y-%m-%d %H:%M:%S")
)
interruption_times = interruption_df["interruption_time"].to_list()


# aggregates event df to structure for task df
aggregated_df = (
    events_df.filter(
        pl.col("timestamp").is_not_null() & pl.col("subject_id").is_not_null()
    )
    .sort(["subject_id", "timestamp"])  # Sort by subject_id and timestamp
    .with_columns([
        # Ensure the 'timestamp' column is correctly cast to datetime[μs] if it's not already
        pl.col("timestamp").cast(pl.Datetime(time_unit="us")).alias("timestamp")
    ])
    .groupby("subject_id")  # Group by subject_id
    .agg([
        pl.col("timestamp").min().alias("start_time"),  # Get the min timestamp as start_time
        pl.col("timestamp").max().alias("end_time"),    # Get the max timestamp as end_time
    ])
    .collect()  # Collect the result into a DataFrame
)

def has_interruption_in_window(end_time):
    for time in interruption_times:
        if end_time <= time <= (end_time + timedelta(days=prediction_window_days)):
            return 1
    return 0

task_df = aggregated_df.with_columns([
    pl.col("end_time").apply(has_interruption_in_window).alias("label")
])
task_df

subject_id,start_time,end_time,label
u32,datetime[μs],datetime[μs],i64
201171,2020-10-06 12:11:01,2020-10-07 05:41:14,0
201172,2020-10-08 07:48:21,2020-10-10 13:16:56,0
201173,2020-10-11 11:32:57,2020-10-13 14:55:12,0
201174,2020-10-14 05:11:08,2020-10-16 14:37:09,0
201175,2020-10-17 05:38:55,2020-10-19 10:46:58,0
201176,2020-10-20 06:00:03,2020-10-22 13:35:38,0
201177,2020-10-23 09:56:49,2020-10-25 14:11:15,1
201178,2020-10-26 11:10:38,2020-10-28 23:09:48,1
201179,2020-10-29 06:55:57,2020-10-31 06:39:27,1


In [10]:
# subject_with_0 = task_df.filter(pl.col("label") == 0)


# select last 100 rows to get statistics for held-out set
last_100_rows = task_df.tail(100)
print(sum(last_100_rows["label"])/len(last_100_rows))
# print(subject_with_0)

0.21


In [28]:
# Save the task_df
  
task_df.write_parquet(TASK_DF_DIR / "task_df_giga_mind_interruption_7_day.parquet") # HÅLL KOLL PÅ SHIFTED ELLER EJ!
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/giga_mind_final/task_dfs


### TIME-TO-INTERRUPTION

In [27]:
COHORT_NAME = "eneryield2"
PROJECT_DIR = Path('/home/filip-marcus/ESGPT_new/EventStreamGPT')
DATA_DIR = PROJECT_DIR / "data/processed" / COHORT_NAME
assert DATA_DIR.is_dir()

TASK_DF_DIR = DATA_DIR / "task_dfs"
TASK_DF_DIR.mkdir(exist_ok=True, parents=False) 

ESD = Dataset.load(DATA_DIR)

events_df = ESD.events_df.lazy()
print(events_df.collect().unique())

Loading events from /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/eneryield2/events_df.parquet...
shape: (311_650, 4)
┌──────────┬────────────┬─────────────────────┬────────────┐
│ event_id ┆ subject_id ┆ timestamp           ┆ event_type │
│ ---      ┆ ---        ┆ ---                 ┆ ---        │
│ u32      ┆ u16        ┆ datetime[μs]        ┆ cat        │
╞══════════╪════════════╪═════════════════════╪════════════╡
│ 2        ┆ 0          ┆ 2025-01-30 14:03:56 ┆ event      │
│ 22       ┆ 0          ┆ 2025-01-31 13:33:52 ┆ event      │
│ 30       ┆ 0          ┆ 2025-02-01 13:33:11 ┆ event      │
│ 35       ┆ 0          ┆ 2025-02-02 13:21:17 ┆ event      │
│ …        ┆ …          ┆ …                   ┆ …          │
│ 311644   ┆ 1358       ┆ 2020-10-06 12:11:11 ┆ event      │
│ 311648   ┆ 1358       ┆ 2020-10-07 02:47:15 ┆ event      │
│ 311652   ┆ 1358       ┆ 2020-10-08 08:09:50 ┆ event      │
│ 311654   ┆ 1359       ┆ 2020-10-06 12:11:02 ┆ event      │
└──────────┴───

In [29]:
import pandas as pd
dataset_0 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_0.csv')
dataset_1 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_1.csv')
dataset_2 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_2.csv')
dataset_3 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_3.csv')
dataset_4 = pd.read_csv('/home/filip-marcus/preprocessed_data/synthetic/pretrain/synthetic_dataset_4.csv')
dataset_5 = pd.read_csv('/home/filip-marcus/preprocessed_data/eneryield/pretrain/eneryield_dataset_1.csv')
dataset_6 = pd.read_csv('/home/filip-marcus/preprocessed_data/eneryield/pretrain/eneryield_dataset_2.csv')


interruption_indices = dataset_6.index[dataset_6['event_label'] == 'interruption'].tolist()                 # ÄNDRA BEROENDE PÅ COHORT!


# Extract the interruption times as strings from the DataFrame (index 3 refers to the 4th column)  
interruption_times_str = dataset_6.iloc[interruption_indices, 3].tolist()                                   # Kan behöva ändra 2, 3 eller 4

datetime_list = pd.to_datetime(interruption_times_str).tolist()
# Step 2: Create a Polars Series with the correct datetime format (microsecond precision)
# Polars can handle datetime objects directly
interruption_times_series = pl.from_pandas(pd.Series(datetime_list))
interruption_times_series = interruption_times_series.cast(pl.Datetime(time_unit="us"))
print(datetime_list)

[Timestamp('2024-12-14 18:14:49'), Timestamp('2024-12-14 18:51:34'), Timestamp('2024-12-14 19:24:22'), Timestamp('2024-12-14 19:26:15'), Timestamp('2024-12-14 20:33:18'), Timestamp('2024-12-14 22:13:54'), Timestamp('2024-12-14 18:14:49'), Timestamp('2024-12-14 18:51:34'), Timestamp('2024-12-14 19:24:22'), Timestamp('2024-12-14 19:26:15'), Timestamp('2024-12-14 20:33:18'), Timestamp('2024-12-14 22:13:54'), Timestamp('2024-12-14 18:14:49'), Timestamp('2024-12-14 18:51:34'), Timestamp('2024-12-14 19:24:22'), Timestamp('2024-12-14 19:26:15'), Timestamp('2024-12-14 20:33:18'), Timestamp('2024-12-14 22:13:54'), Timestamp('2024-12-14 18:14:49'), Timestamp('2024-12-14 18:51:34'), Timestamp('2024-12-14 19:24:22'), Timestamp('2024-12-14 19:26:15'), Timestamp('2024-12-14 20:33:18'), Timestamp('2024-12-14 22:13:54'), Timestamp('2024-11-12 09:18:51'), Timestamp('2024-11-12 09:18:51'), Timestamp('2024-11-12 09:18:51'), Timestamp('2024-11-12 09:18:51'), Timestamp('2024-10-26 12:44:31'), Timestamp('20

In [30]:
from bisect import bisect_right

# Ensure datetime_list is sorted.
datetime_list.sort()

# Group events_df by subject_id to get start_time (min) and end_time (max)
subject_df = (
    events_df
    .groupby("subject_id")
    .agg([
        pl.col("timestamp").min().alias("start_time"),
        pl.col("timestamp").max().alias("end_time")
    ])
).collect()

# Define a function that, given an end_time, computes time-to-next interruption (in minutes)
def compute_time_to_next_interrupt(end_time):
    # bisect_right returns the index where end_time would be inserted to keep datetime_list sorted.
    idx = bisect_right(datetime_list, end_time)
    if idx < len(datetime_list):
        next_interrupt = datetime_list[idx]
        # Compute difference in minutes
        diff_minutes = (next_interrupt - end_time).total_seconds() / (60.0 * 60.0 * 24.0)
        return diff_minutes
    else:
        # Return None if no interruption occurs after end_time
        return None

# Apply the function row-wise to create the "label" column.
tti_df = subject_df.with_columns(
    pl.col("end_time").apply(compute_time_to_next_interrupt).alias("label")
)
tti_df = tti_df.sort(by="subject_id")
tti_df = tti_df.drop_nulls(subset=["label"])
tti_df

subject_id,start_time,end_time,label
u16,datetime[μs],datetime[μs],f64
50,2024-12-12 05:12:19,2024-12-13 14:49:01,1.142917
51,2024-12-09 06:49:17,2024-12-12 19:44:33,1.937685
52,2024-12-09 06:49:17,2024-12-09 13:13:17,5.209398
53,2024-12-07 12:37:51,2024-12-09 13:13:17,5.209398
54,2024-12-06 12:16:05,2024-12-09 13:13:17,5.209398
55,2024-12-05 02:07:18,2024-12-07 12:37:51,7.234005
56,2024-12-04 01:47:35,2024-12-07 12:37:51,7.234005
57,2024-12-03 12:22:42,2024-12-06 15:02:06,8.133831
58,2024-12-02 11:45:54,2024-12-05 19:15:57,8.957546


In [31]:
import numpy as np
avg_value = tti_df.select(pl.col("label").mean()).to_dict()["label"][0]
mse = tti_df.select(((pl.col("label") - avg_value) ** 2).mean()).to_dict()["label"][0]
mae = tti_df.select((pl.col("label") - avg_value).abs().mean()).to_dict()["label"][0]

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")

Mean Squared Error: 627.0564204498098
Mean Absolute Error: 20.419509255296575


In [32]:
tti_df.write_parquet(TASK_DF_DIR / "task_df_eneryield_tti.parquet")
print(f"saved task_df to {TASK_DF_DIR}")

saved task_df to /home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/eneryield2/task_dfs


In [45]:
import pyarrow.parquet as pq

table = pq.read_table('/home/filip-marcus/ESGPT_new/EventStreamGPT/data/processed/eneryield1/DL_reps/held_out_0.parquet')
df = table.to_pandas()
pd.set_option('display.max_rows', None)
df

,subject_id,static_measurement_indices,static_indices,start_time,time,dynamic_measurement_indices,dynamic_indices,dynamic_values
0,345,[2],[3],2024-03-30 12:31:40,"[0.0, 31.233333333333334, 193.41666666666666, ...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.02374060507167206, -0.029759762..."
1,346,[2],[3],2024-04-02 02:01:51,"[0.0, 5.733333333333333, 16.016666666666666, 1...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.0243695474844895, -0.0308550587..."
2,347,[2],[3],2024-04-05 00:13:46,"[0.0, 79.6, 83.95, 88.8, 93.25, 107.2833333333...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.02445576296819442, -0.030506755..."
3,348,[2],[3],2024-04-08 04:34:54,"[0.0, 1.9, 2.066666666666667, 2.11666666666666...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.026544832812788254, -0.03017534..."
4,349,[2],[3],2024-04-11 03:27:45,"[0.0, 6.05, 73.06666666666666, 73.216666666666...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.027239847572272678, -0.02830178..."
5,350,[2],[3],2024-04-14 05:42:04,"[0.0, 0.7333333333333333, 5.25, 1248.9, 1249.8...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 7, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.026456561178063505, -0.02725922..."
6,351,[2],[3],2024-04-17 10:42:57,"[0.0, 812.8166666666667, 1185.8333333333333, 1...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.025098144343260572, -0.02943949..."
7,352,[2],[3],2024-04-20 05:24:18,"[0.0, 0.5166666666666667, 2923.15, 2939.666666...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.025110731252853426, -0.02777101..."
8,353,[2],[3],2024-04-23 02:51:46,"[0.0, 33.983333333333334, 34.03333333333333, 2...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.02466890878706391, -0.027083813..."
9,354,[2],[3],2024-04-28 05:02:24,"[0.0, 155.6, 155.9, 156.05, 156.35, 156.9, 164...","[[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, ...","[[1, 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24...","[[nan, nan, -0.02460722425646902, -0.030025789..."
